# Mount the Google *DRIVE*

---



In [ ]:
'''
Mount the GDrive containing GitHub local repository to Python environment
'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Functions -- Load JSONs on GDrive to DataFrames**

In [ ]:
'''
  Functions to load data from JSON files on GDrive and map them to dataframes
  for analysis.
'''
import os
import json
import pandas as pd

#is_print=True
# Get absolute paths to all files in a directory
def get_abs_fpaths(dir):
    for dirpath,_,filenames in os.walk(dir):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))

# Load a JSON data file into a dataframe,given the full path
def load_file (f):
  try:
    df = pd.read_json(f)
    df.drop(columns=['success','code'])
    return df
  except:
    raise Exception(f'''File not found: {f}''')

# Decode common record attributes from a JSON file
def load_common(f, report_type, retd):
  #print('''decode_common() - Start ''')
  state=None
  year=None
  quarter=None

  if 'state' in f:
    state,year,quarter = os.path.normpath(f).split(os.sep)[-3:]
  else:
    year,quarter=os.path.normpath(f).split(os.sep)[-2:]
  quarter=quarter.split('.')[0]
  geo_type='CON'
  geo_name='india'
  geo_parent=None
  if year is not None and quarter is not None:
    if state is not None:
      geo_type='STA'
      geo_name=state
      geo_parent='india'

  #print('''decode_common() - End ''')
  retd['year'].append(year)
  retd['quarter'].append(quarter)
  retd['geo_type'].append(geo_type)
  retd['geo_name'].append(geo_name)
  retd['geo_parent'].append(geo_parent)

  if report_type == 'top':
    retd['top_in'].append(geo_type)

  return

def load_top_txns(f,report_type,retd):
  #print('load_top_txns() - Start')
  # Load JSON contents into a temp dataframe
  df=load_file(f)

  # Seek to the contents of the 'transaction' data
  txn_states = df.loc['states','data']
  txn_districts = df.loc['districts','data']
  txn_pincodes = df.loc['pincodes','data']
  if txn_states is not None:
    for txn in txn_states:
      load_common(f, report_type, retd)
      #retd['category'].append(None)
      retd['geo_parent'][-1]='india'
      retd['geo_type'][-1]='STA'
      retd['geo_name'][-1]=txn['entityName']
      retd['stat_type'].append(txn['metric']["type"])
      retd['count'].append(txn['metric']["count"])
      retd['amount'].append(txn['metric']["amount"])

  if txn_districts is not None:
    for txn in txn_districts:
      load_common(f, report_type, retd)
      #retd['category'].append(None)
      retd['geo_parent'][-1]=retd['geo_name'][-1]
      retd['geo_type'][-1]='DIS'
      retd['geo_name'][-1]=txn['entityName']
      retd['stat_type'].append(txn['metric']["type"])
      retd['count'].append(txn['metric']["count"])
      retd['amount'].append(txn['metric']["amount"])

  if txn_pincodes is not None:
    for txn in txn_pincodes:
      load_common(f, report_type, retd)
      #retd['category'].append(None)
      retd['geo_parent'][-1]=retd['geo_name'][-1]
      retd['geo_type'][-1]='PIN'
      retd['geo_name'][-1]=txn['entityName']
      retd['stat_type'].append(txn['metric']["type"])
      retd['count'].append(txn['metric']["count"])
      retd['amount'].append(txn['metric']["amount"])

  return

def load_hover_txns(f,report_type,retd):
  print('load_hover_txns() - Start Line 96')
  # Load JSON contents into a temp dataframe
  df=load_file(f)
  # Seek to the contents of the 'hover transaction' data
  hover_recs = df.loc['hoverDataList','data']
  if hover_recs is not None:
    for hover_rec in hover_recs:
      load_common(f, report_type, retd)
      print(hover_rec)
      retd['category'].append(hover_rec["name"])
      retd['stat_type'].append(hover_rec['metric']["type"])
      retd['count'].append(hover_rec['metric']["count"])
      retd['amount'].append(hover_rec['metric']["amount"])

  return None
#  Decode 'transaction' record

def load_agg_txns(f,report_type,retd):
  #print('load_agg_txns() - Start')

  # Load JSON contents into a temp dataframe
  df=load_file(f)

  # Seek to the contents of the 'transaction' data
  txn_recs = df.loc['transactionData','data']
  for txn_rec in txn_recs:
    # Collect the common fields from path name
    load_common(f, report_type, retd)

    # Process record for all category
    for payment_rec in txn_rec['paymentInstruments']:
      retd['category'].append(txn_rec["name"])
      retd['stat_type'].append(payment_rec["type"])
      retd['count'].append(payment_rec["count"])
      retd['amount'].append(payment_rec["amount"])

  #print('decode_txn() - End')
  return

def load_top_users(f,report_type,retd):
  # Load JSON contents into a temp dataframe
  df=load_file(f)

  # Seek to the contents of the 'transaction' data
  user_states = df.loc['states','data']
  user_districts = df.loc['districts','data']
  user_pincodes = df.loc['pincodes','data']

  if user_states is not None:
    for user in user_states:
      load_common(f,report_type,retd)
      retd['geo_parent'][-1]='india'
      retd['geo_type'][-1]='STA'
      retd['geo_name'][-1]=user['name']
      retd['reg_users'].append(user['registeredUsers'])

  if user_districts is not None:
    for user in user_districts:
      load_common(f,report_type,retd)
      if retd['top_in'][-1] == 'STA':
        retd['geo_parent'][-1]=retd['geo_name'][-1]
      retd['geo_type'][-1]='DIS'
      retd['geo_name'][-1]=user['name']
      retd['reg_users'].append(user['registeredUsers'])

  if user_pincodes is not None:
    for user in user_pincodes:
      load_common(f,report_type,retd)
      if retd['top_in'][-1] == 'STA':
        retd['geo_parent'][-1]=retd['geo_name'][-1]
      retd['geo_type'][-1]='PIN'
      retd['geo_name'][-1]=user['name']
      retd['reg_users'].append(user['registeredUsers'])


# Decode 'user' record
def load_agg_users(f,record_class,retd):
  #print('decode_user - Start')

  # Load JSON contents into a temp dataframe
  df=load_file(f)

  # Seek to the contents of the 'user.aggregated' data
  user_stat_rec = df.loc['aggregated','data']

  #print(f)
  #print (df.to_markdown())
  # Seek to the contents of the 'user.device' data
  device_recs = df.loc['usersByDevice','data']

  # For each device row populate both aggregate and device fields
  if device_recs is not None:
    for device_rec in device_recs:
      # Collect the common fields from path name
      load_common(f, record_class,retd)
      retd['reg_users'].append(user_stat_rec['registeredUsers'])
      retd['app_opens'].append(user_stat_rec['appOpens'])
      retd['brand'].append(device_rec['brand'])
      retd['count'].append(device_rec['count'])
      retd['percentage'].append(device_rec['percentage'])
  else:
    load_common(f, record_class,retd)
    retd['reg_users'].append(user_stat_rec['registeredUsers'])
    retd['app_opens'].append(user_stat_rec['appOpens'])
    retd['brand'].append('Unknown')
    retd['count'].append(0)
    retd['percentage'].append(100)

  #print('decode_user - End')
  return

def load_hover_users(f,record_class,retd):
  return None

def load_top_ins(f,report_type,retd):
  # Load JSON
  df=load_file(f)

  # Seek to the contents of the 'transaction' data
  ins_states = df.loc['states','data']
  ins_districts = df.loc['districts','data']
  ins_pincodes = df.loc['pincodes','data']

  if ins_states is not None:
    for ins in ins_states:
      load_common(f,report_type,retd)
      retd['geo_parent'][-1]='india'
      retd['geo_type'][-1]='STA'
      retd['geo_name'][-1]=ins['entityName']
      retd['stat_type'].append(ins['metric']["type"])
      retd['count'].append(ins['metric']["count"])
      retd['amount'].append(ins['metric']["amount"])

    if ins_districts is not None:
      for ins in ins_districts:
        load_common(f,report_type,retd)
        if retd['top_in'][-1] == 'STA':
          retd['geo_parent'][-1]=retd['geo_name'][-1]
        retd['geo_type'][-1]='DIS'
        retd['geo_name'][-1]=ins['entityName']
        retd['stat_type'].append(ins['metric']["type"])
        retd['count'].append(ins['metric']["count"])
        retd['amount'].append(ins['metric']["amount"])

    if ins_pincodes is not None:
      for ins in ins_pincodes:
        load_common(f,report_type, retd)
        if retd['top_in'][-1] == 'STA':
          retd['geo_parent'][-1]=retd['geo_name'][-1]
        retd['geo_type'][-1]='PIN'
        retd['geo_name'][-1]=ins['entityName']
        retd['stat_type'].append(ins['metric']["type"])
        retd['count'].append(ins['metric']["count"])
        retd['amount'].append(ins['metric']["amount"])

# Decode 'insurance' record
def load_agg_ins(f,report_type,retd):
  #print('decode_ins - Start')

  # Load JSON contents into a temp dataframe
  df=load_file(f)

  # print(df.to_markdown())
  # Seek to the contents of the 'insurance' data
  ins_recs = df.loc['transactionData','data']

  # For each 'insurance' row populate fields
  for ins_rec in ins_recs:
    # Collect the common fields from path name
    load_common(f, report_type, retd)
    for payment_rec in ins_rec['paymentInstruments']:
      retd['category'].append(ins_rec["name"])
      retd['stat_type'].append(payment_rec["type"])
      retd['count'].append(payment_rec["count"])
      retd['amount'].append(payment_rec["amount"])
  #print('decode_ins - End')
  return

def load_hover_ins(f,report_type,retd):
  return None

# Supported record types
record_types = {
  "transaction":{
    "report_type":{
      "aggregated":{'decoder':load_agg_txns,"columns":['year','quarter', 'geo_type', 'geo_name', 'geo_parent', 'category', 'stat_type', 'count', 'amount']},
      "top":{'decoder':load_top_txns,'columns':['year','quarter', 'geo_type', 'geo_name', 'geo_parent', 'top_in', 'stat_type', 'count', 'amount']},
      #"map":{'decoder':load_hover_txns,"columns":['year','quarter', 'geo_type', 'geo_name', 'geo_parent', 'category', 'stat_type', 'count', 'amount']}
    }
  },

  "user":{
    "report_type":{
       "aggregated":{'decoder':load_agg_users,"columns":['year','quarter', 'geo_type', 'geo_name', 'geo_parent', 'reg_users', 'app_opens','brand','count','percentage'],},
       "top":{'decoder':load_top_users,'columns':['year','quarter', 'geo_type', 'geo_name', 'geo_parent','top_in','reg_users']},
       #"map":{'decoder':load_hover_users,"columns":['year','quarter', 'geo_type', 'geo_name', 'geo_parent', 'reg_users', 'app_opens','brand','count','percentage'],}
    }
  },

  "insurance":{
    "report_type":{
      "aggregated":{'decoder':load_agg_ins,"columns":['year','quarter', 'geo_type', 'geo_name', 'geo_parent','category', 'stat_type', 'count', 'amount'],},
      "top":{'decoder':load_top_ins,'columns':['year','quarter', 'geo_type', 'geo_name','geo_parent', 'top_in', 'stat_type', 'count', 'amount']},
      #"map":{'decoder':load_hover_ins,"columns":['year','quarter', 'geo_type', 'geo_name','geo_parent', 'category', 'stat_type', 'count', 'amount'],}
     }
  }
}

# Supported record classes
#report_type = {
#    "aggregated":"Aggregated",
#    "hover": "Hover",
#    "top": "Top"
#}

def get_abs_fpaths(dir):
  for dirpath,_,filenames in os.walk(dir):
    for f in filenames:
      yield os.path.abspath(os.path.join(dirpath, f))

# Load data
def load_data(root_dir):

  print( '''load_data() - Start''')
  # Dictionary of resulting dataframes
  ret = dict()

  # Get aggregate, top and map for each record type
  for rec_type in record_types.keys(): # trans, user, insurance
    ret[rec_type]=dict()
    #print(f'''Loading {rec_type} data''')
    # Do this for every report_type - aggregate, top and map/hover
    for report_type in record_types[rec_type]['report_type'].keys():
      # Set the columns of output dataframe
      columns = record_types[rec_type]['report_type'][report_type]['columns']
      #print(f'''Loading {report_type} data''')

      # Create an output dataframe to store record's from multiple files
      #ret_df = pd.DataFrame(columns)
      retd = dict()
      for c in columns:
        retd[c]=list()

      # Add the output dataframe to the output dictionary of dataframes
      ret[rec_type][report_type]=retd

      # Construct absolute path to folder containing files for
      # a record class and type
      full_path=f'''{root_dir}/{report_type}/{rec_type}'''
      if report_type == 'map':
        full_path += '/hover'
      # Decode contents of each file accumulate in a dataframe
      count=0
      print({full_path})
      for f in get_abs_fpaths(full_path):
        loader = record_types[rec_type]['report_type'][report_type]['decoder']
        loader(f,report_type,retd)
        count +=1

  print( f'''load_data() {count} files processed - End''')
  return ret

  '''
  Helper functions to pre-process 'transaction' data for DB writes
'''
import sqlalchemy
from sqlalchemy import create_engine


def get_geo_info(connection):
  sql = '''SELECT id,name,geo_type,parent_id FROM geo;'''
  df=pd.read_sql_query(sql, connection)
  return df

def lookup_geo_id(geo_type, geo_name, geo_parent,geo):
  if geo_type == 'STA':
    geo_name=geo_name.replace(' ','-')
  if geo_name == 'nan':
    print(f'''geo_type = {geo_type}, geo_parent = {geo_parent}''')
  #print(f'''geo_type:{geo_type}, geo_name:{geo_name}''')
  geo_entries = geo[(geo['geo_type']==geo_type) & (geo['name']==geo_name)].reset_index()
  if geo_entries.shape[0] == 1:
    return geo_entries.loc[0,'id']
  elif geo_entries.shape[0]>1:
    if geo_parent is not None:
      geo_parents = geo[geo['name']==geo_parent].reset_index()
      if geo_parents.shape[0] > 0:
        return geo_entries[geo_entries['parent_id']==geo_parents.loc[0,'id']].reset_index().loc[0,'id']
  return -1

def preprocess_generic(df,geo):
  df['geo_name']=df['geo_name'].astype(str)

  # Lookup the geo_id from type and name
#  df['geo_typename']=df['geo_type'] + ',' + df['geo_name']
#  df['geo_id']=df['geo_typename'].apply(lookup_geo_id, args=(geo,))
  df['geo_id']=df.apply(lambda x: lookup_geo_id(x['geo_type'],x['geo_name'],x['geo_parent'],geo),axis=1)
  # Drop unnecessary columns
  df.drop(columns=['geo_type', 'geo_name', 'geo_parent'], inplace=True)
  df = df.reset_index().rename(columns={'index':'id'})
  return df

def save_txns(agg_txns,top_txns,conn):
  # Lookup Geo_id and drop geo_type and geo_name columns
  # Step 1: Replace geo_type & geo_name with geo_id as a single column
  # Step 2: Add id column
  geo=get_geo_info(conn)

  # Review the shape
  print(f'''geo - {geo.shape}''')

  agg_txns = preprocess_generic(agg_txns,geo)
  top_txns = preprocess_generic(top_txns,geo)
  print(f'''agg_txns(db): {agg_txns.shape}, top_txns(db):{top_txns.shape}''')
  #print(f'''{top_txns[top_txns['geo_id']==-1].to_markdown()}''')
  # Write to 'transaction_*' tables
  agg_txns.to_sql(con=conn, name='transaction_agg',if_exists='append',index=False)
  top_txns.to_sql(con=conn, name='transaction_top',if_exists='append',index=False)

  conn.commit()

def save_ins(agg_ins,top_ins,conn):
  # Lookup Geo_id and drop geo_type and geo_name columns
  # Step 1: Replace geo_type & geo_name with geo_id as a single column
  # Step 2: Add id column
  geo=get_geo_info(conn)

  # Review the shape
  print(f'''geo - {geo.shape}''')

  agg_ins = preprocess_generic(agg_ins,geo)
  top_ins = preprocess_generic(top_ins,geo)
  print(f'''agg_ins(db): {agg_ins.shape}, top_ins(db):{top_ins.shape}''')

  # Write to 'insurance*' tables
  agg_ins.to_sql(con=conn, name='insurance_agg',if_exists='append',index=False)
  top_ins.to_sql(con=conn, name='insurance_top',if_exists='append',index=False)

  conn.commit()

def save_users(agg_users,top_users,conn):
  # Lookup Geo_id and drop geo_type and geo_name columns
  # Step 1: Replace geo_type & geo_name with geo_id as a single column
  # Step 2: Add id column
  geo=get_geo_info(conn)

  # Review the shape
  print(f'''geo - {geo.shape}''')

  agg_users = preprocess_generic(agg_users,geo)
  top_users = preprocess_generic(top_users,geo)
  print(f'''agg_users(db): {agg_users.shape}, top_users(db):{top_users.shape}''')

  u_agg = agg_users[['year','quarter','geo_id','reg_users','app_opens']]
  d_agg = agg_users[['year','quarter','geo_id','brand','count','percentage']].reset_index().rename(columns={'index':'id'})
  u_agg.drop_duplicates(inplace=True,ignore_index=True)

  u_agg = u_agg.reset_index().rename(columns={'index':'id'})
  u_agg['stat_type']='TOTAL'
  top_users['stat_type']='TOTAL'

  # Write to 'users*' tables
  u_agg.to_sql(con=conn, name='user_agg',if_exists='append',index=False)
  d_agg.to_sql(con=conn, name='device_agg',if_exists='append',index=False)
  top_users.to_sql(con=conn, name='user_top',if_exists='append',index=False)

  conn.commit()
  print(f'''user_agg(db): {u_agg.shape}, device_agg(db): {d_agg.shape}, top_users(db):{top_users.shape}''')

'''
  Helper functions to save Geo
'''

def lookup_state_id(state_name, states):
    ret=None
    ret = states[states['name']==state_name].reset_index()['id'][0]
    return ret;

def save_geo(states,districts,pincodes,conn):
    '''
      Prepare the 'states' DF for DB insertion
    '''
    # Replace the DF column names to match the DB column names
    states.reset_index(inplace=True)
    states.rename(columns={'index':'id', 'parent':'parent_id'},inplace=True)

    # Add geo_type column and set the value to 'STA'
    states['geo_type']='STA'

    # Assign non-conflicting IDs for DB rows
    states.id=states.id+1

    '''
      Prepare the 'districts' DF for DB insertion
    '''
    # Replace the DF column names to match the DB column names
    districts.reset_index(inplace=True)
    districts.rename(columns={'index':'id', 'parent':'parent_id'},inplace=True)

    # Add geo_type column and set the value to 'DIS'
    districts['geo_type']='DIS'
    districts['name']=districts['name'].apply(lambda x: x.split('|')[0] )

    # Assign non-conflicting DB ids for the districts
    districts.id=districts.id+len(states.index)+1

    '''
      Prepare the 'pincodes' DF for DB insertion
    '''
    # Replace the DF column names to match the DB column names
    pincodes.reset_index(inplace=True)
    pincodes.rename(columns={'index': 'id', 'parent':'parent_id'},inplace=True)

    # Add geo_type column and set the value to 'PIN'
    pincodes['geo_type']='PIN'

    # Assign non-conflicting DB ids for the districts
    pincodes.id=pincodes.id+len(states.index)+ len(districts.index)+1

    # Impute null values (NOTE: Analyis showed that - the only NA value corrsponded to 000000 for parent state ID=18)
    pincodes['name']=pincodes.name.fillna('nan')

    '''
      Replace parent names with IDs assigned in previous steps
    '''
    states['parent_id']=0
    districts['parent_id']=districts.apply(lambda x: lookup_state_id(x['parent_id'],states),axis=1)
    pincodes['parent_id']=pincodes.apply(lambda x: lookup_state_id(x['parent_id'],states),axis=1)

    # Write to the DB
    country_df=pd.DataFrame({'id':0,'geo_type':'CON','name':'india','parent_id':None},index=[0])
    country_df.to_sql(con=conn, name='geo',if_exists='append',index=False)
    states.to_sql(con=conn, name='geo',if_exists='append',index=False)
    districts.to_sql(con=conn, name='geo',if_exists='append',index=False)
    pincodes.to_sql(con=conn, name='geo',if_exists='append',index=False)
    conn.commit()

'''
  Index Geo information from data
'''

def index_geo_info(datadir):
  ret = dict()
  ret['states'] = dict()
  ret['districts'] = dict()
  ret['pincodes'] = dict()

  for f in get_abs_fpaths(datadir):
    state=None
    if 'state' in f:
      state= os.path.normpath(f).split(os.sep)[-3]
      # Index the state
      if state not in ret['states']:
        ret['states'][state]='india'

      if 'top' in f:
        # Load JSON contents into a temp dataframe
        df=load_file(f)

        # Seek to the contents of the 'transaction' data
        districts = df.loc['districts','data']
        pincodes = df.loc['pincodes','data']

        # Index the districts in the file
        if districts is not None:
          for entry in districts:
            if 'entityName' in entry:
              district = entry['entityName']
            elif 'name' in entry:
              district = entry['name']
            if district not in ret['districts']:
              ret['districts'][district] = state
            else:
              if ret['districts'][district] != state:
                ret['districts'][district+"|"+state]=state

        # Index the pincodes in the file
        if pincodes is not None:
          for entry in pincodes:
            if 'entityName' in entry:
              pincode = entry['entityName']
            elif 'name' in entry:
              pincode = entry['name']
            if pincode not in ret['pincodes']:
              ret['pincodes'][pincode] = state
  return ret

# **Functions to Fetch DASHBOARDS**

In [ ]:
'''
  Get Dashboards from mySQL db
'''
import sqlalchemy
import pandas as pd

def convert_to_geojson_state(name):
	if name is None:
		return name
	return ' '.join([x.capitalize() for x in name.split('-')])


def get_years_list():
  return [2018,2019,2020,2021,2022,2023,2024]

def get_quarters_list():
  return [(1,'Q1'),(2,'Q2'),(3,'Q3'),(4,'Q4')]

def get_states_list(conn):
  sql= f'''SELECT g.id as 'id', g.name as 'State' FROM geo g WHERE geo_type='STA' ORDER BY name ASC'''
  # Get states from db
  stmt = sqlalchemy.text(sql,)
  df=pd.read_sql_query(stmt, conn)
  return df


def fetch_txn_dash(conn, year,quarter, state=None):
  # dashboard dict for output
  dashboard=dict()
  dashboard['object']='Transaction'
  dashboard['topic']='Transaction Details'
  dashboard['description']='Transaction Details for selected timeline'
  dashboard['grand_total']=dict()
  grand_total=dashboard['grand_total']
  grand_total['name']='Transaction Summary'
  grand_total['charts']=list()

  # Detailed Totals   -- Sub dict() being created
  dashboard['det_total']=dict()
  det_total=dashboard['det_total']
  det_total['name']='Payment Modes'
  det_total['charts']=list()

  # SQLs and Filter Procesing
  sql= f'''SELECT t.category as 'Payment Mode', sum(t.count) as 'Transaction Count', sum(t.amount) 'Transaction Amount' FROM transaction_agg t WHERE year=:year'''

  # Extend the WHERE clause based on other fiters
  params=dict()
  params['year']=year
  if quarter is not None:
    sql += ' AND quarter=:quarter'
    params['quarter']=quarter[0]

  sql += ' GROUP BY t.category'

  # Execute agg SQL to get df with 4 rows and 3 columns
  stmt = sqlalchemy.text(sql,)
  df=pd.read_sql_query(stmt, conn, params=params)

  # Calculating Grand Totals for Amount/Count from df and store in dashboard of ['grand_total' & 'det_total']
  grand_ct_total=dict()
  grand_ct_total['title']='Transaction Count'
  grand_ct_total['value']=0

  grand_amt_total=dict()
  grand_amt_total['title']='Transaction Amount (Rs)'
  grand_amt_total['value'] = 0
  for idx, row in df.iterrows():
    det_ct=dict()
    det_ct['title']=row['Payment Mode']               #  for count metric widget
    det_ct['value']=row['Transaction Count']          #  for count metric widget
    det_ct['metric']='Transaction Count'              #  for count metric widget
    det_total['charts'].append(det_ct)
    grand_ct_total['value'] += row['Transaction Count']             # grand total of count


    det_amt=dict()
    det_amt['title']=row['Payment Mode']
    det_amt['value']=row['Transaction Amount']
    det_amt['metric']='Transaction Amount'
    det_total['charts'].append(det_amt)
    grand_amt_total['value'] += row['Transaction Amount']

  grand_total['charts'].append(grand_ct_total)
  grand_total['charts'].append(grand_amt_total)

  # Construct TOP Dashboards
  if( quarter is not None):
    dashboard['top']=dict()
    sql_top=f'''SELECT g.name as 'Geo', g.geo_type as 'Geo Type', t.count as 'Txn Count' ,t.amount as 'Txn Amount (Rs)' FROM transaction_top t JOIN geo g ON (t.geo_id=g.id) WHERE year=:year AND quarter=:quarter AND top_in=:top_in'''
    params_top=dict()
    params_top['year']=year
    params_top['quarter']=quarter[0]

    if state is not None:
      params_top['top_in']='STA'
      sql_top += ' AND geo_id IN (SELECT id from geo WHERE parent_id=:parent_id)'
      params_top['parent_id']=state
    else:
      params_top['top_in']='CON'

    stmt = sqlalchemy.text(sql_top,)
    df_top=pd.read_sql_query(stmt, conn, params=params_top)

    # Populate 'top'
    top=dashboard['top']
    top['name']= 'Txns By Top Geo'
    top['charts']=list()

    top['charts'].append(dict())
    top['charts'][0]['title']='Top States'
    top['charts'][0]['data']=df_top[df_top['Geo Type']=='STA'].reset_index().drop(columns=['index']).sort_values('Txn Count', ascending=False)

    top['charts'].append(dict())
    top['charts'][1]['title']='Top Districts'
    top['charts'][1]['data']=df_top[df_top['Geo Type']=='DIS'].reset_index().drop(columns=['index']).sort_values('Txn Count', ascending=False)

    top['charts'].append(dict())
    top['charts'][2]['title']='Top Pincodes'
    top['charts'][2]['data']=df_top[df_top['Geo Type']=='PIN'].reset_index().drop(columns=['index']).sort_values('Txn Count', ascending=False)

  # Map Data
  if state is None:
    dashboard['by_geo']=dict()
    by_geo = dashboard['by_geo']
    by_geo['name']='Insurance By State'

    params_states=dict()
    params_states['year']=year
    sql_states = f'''SELECT S1.geo_id AS 'State Code', g.name AS 'State', S1.counts AS 'Txn Count', S1.amount AS 'Txn Amount' FROM (SELECT geo_id, sum(t.count) as counts, sum(t.amount) as amount FROM transaction_agg t WHERE year=:year'''
    if quarter is not None:
      sql_states += ' AND quarter=:quarter'
      params_states['quarter']=quarter[0]

    sql_states += ''' GROUP BY geo_id)S1 JOIN geo g ON (S1.geo_id=g.id) WHERE geo_type='STA' '''

    stmt = sqlalchemy.text(sql_states,)
    df_states=pd.read_sql_query(stmt, conn,params=params_states)
    df_states['State']=df_states['State'].apply(convert_to_geojson_state)
    by_geo['charts']=list()
    by_geo['charts'].append({'title':'Transactions By State','data':df_states})
  return dashboard
'''  {
      'recordType': 'Transaction',
      'topic': 'Transaction Details',
      'description':'Details for selected timeline',
      'grand_total':{'name':'Transaction Summary','charts':[{'title': 'Transaction Count', 'value': 50000},{'title': 'Amount (in Rs.)', 'value': 78500}]},
      'top':{'name':'Transactions By Top Locations','charts':[ {'title': 'Top States', 'data': pd.DataFrame({'State':['tamil-nadu','maharastra', 'kerala','karnataka', 'bihar','manipur','andhra-preadesh','jharkand','jammu-&-kashmir', 'puduchery'], 'Transaction Count':[10,9,8,7,6,5,4,3,2,1], 'Amount':[10000,9000,8000,7000,6000,5000,4000,3000,2000,1000]})},
              {'title': 'Top Districts', 'data': pd.DataFrame({'District':['chengalpet','thiruvallur', 'krishnagiri','ramanathapuram','salem','namakkal','karur','coimbatore','nilgiris','hosur'], 'Transaction Count':[10,9,8,7,6,5,4,3,2,1],'Amount':[10000,9000,8000,7000,6000,5000,4000,3000,2000,1000]})},
              {'title': 'Top Pincodes', 'data': pd.DataFrame({'Pincode':['600023','600001', '600011','600028','600038','600048','600058','600068','600078','600088'], 'Transaction Count':[10,9,8,7,6,5,4,3,2,1], 'Amount':[10000,9000,8000,7000,6000,5000,4000,3000,2000,1000]})}
      ]},
      'trend':{'name':'Quarterly Transaction Trends', 'charts':[{'title': 'Quarterly Transaction Trend', 'data': pd.DataFrame({'Quarter':['Q1','Q2','Q3','Q4'], 'Transaction Count':[1,4,3,2]})}]},
      'det_total':{'name': 'Payment Modes','charts':[{'title':'Recharge & bill payments', 'value': 50000,'metric':'Count'},{'title': 'Peer-to-peer payments', 'value': 78500, 'metric':'Count'}, {'title': 'Merchant payments', 'value': 98500,'metric':'Count'},{'title': 'Financial Services', 'value': 50255,'metric':'Count'},{'title':'Recharge & bill payments', 'value': 50000,'metric':'Amount'},{'title': 'Peer-to-peer payments', 'value': 78500, 'metric':'Amount'}, {'title': 'Merchant payments', 'value': 98500,'metric':'Amount'},{'title': 'Financial Services', 'value': 50255,'metric':'Amount'}]}
      }
'''

def fetch_ins_dash(conn, year,quarter, state=None):
  # Construct the dashboard dictionary
  dashboard=dict()
  dashboard['object']='Insurance Details'
  dashboard['title']='Insurance Details'
  dashboard['description']='Insurance Details for Selected Timeline'
  dashboard['grand_total']=dict()
  grand_total=dashboard['grand_total']
  grand_total['charts']=list()

  # SQLs and Filter Procesing
  sql= f'''SELECT i.count as 'Insurance Count', i.amount as 'Insurance Amount' FROM insurance_agg i WHERE year=:year'''

  params=dict()
  params['year']=year
  # if quarter is specified add to WHERE clause
  if quarter is not None:
    sql += ' AND quarter=:quarter'
    params['quarter']=quarter[0]

  # setting the geo_id filter
  sql += ' AND geo_id=:geo_id'
  if state is not None:
    params['geo_id']=state
  else:
    params['geo_id']=0 #Refers to country India

  # Execute agg SQL
  stmt = sqlalchemy.text(sql,)
  df=pd.read_sql_query(stmt, conn, params=params)

  # Populate 'aggregates' & 'aux_aggregates'
  grand_ct_total=dict()
  grand_ct_total['title']='Insurance Count'
  grand_ct_total['value']=0

  grand_amt_total=dict()
  grand_amt_total['title']='Insurance Amount (Rs)'
  grand_amt_total['value']=0

  for idx, row in df.iterrows():
    grand_ct_total['value'] += row['Insurance Count']
    grand_amt_total['value'] += row['Insurance Amount']

  grand_total['charts'].append(grand_ct_total)
  grand_total['charts'].append(grand_amt_total)

  # Execute Top SQL
  if quarter is not None:
    dashboard['top']=dict()
    sql_top=f'''SELECT g.name as 'Geo', g.geo_type as 'Geo Type', i.count as 'Insurance Count', i.amount as 'Insurance Amount (Rs)' FROM insurance_top i JOIN geo g ON (i.geo_id=g.id) WHERE year=:year AND quarter=:quarter AND top_in=:top_in'''

    params_top=dict()
    params_top['year']=year
    params_top['quarter']=quarter[0]

    if state is not None:
      sql_top += ' AND geo_id IN (SELECT id FROM geo WHERE parent_id=:parent_id)'
      params_top['top_in']='STA'
      params_top['parent_id']=state
    else:
      params_top['top_in']='CON'

    stmt = sqlalchemy.text(sql_top,)
    df_top=pd.read_sql_query(stmt, conn,params=params_top)

    # Populate 'top'
    top=dashboard['top']
    top['name']= 'Insurance By Top Locations'
    top['charts']=list()

    #if state is None:
    top['charts'].append(dict())
    top['charts'][0]['title']='Top States'
    top['charts'][0]['data']=df_top[df_top['Geo Type']=='STA'].reset_index().drop(columns=['index']).sort_values('Insurance Count', ascending=False)

    top['charts'].append(dict())
    #cidx= len(top['charts'])-1
    top['charts'][1]['title']='Top Districts'
    top['charts'][1]['data']=df_top[df_top['Geo Type']=='DIS'].reset_index().drop(columns=['index']).sort_values('Insurance Count', ascending=False)

    top['charts'].append(dict())
    #cidx= len(top['charts'])-1
    top['charts'][2]['title']='Top Pincodes'
    top['charts'][2]['data']=df_top[df_top['Geo Type']=='PIN'].reset_index().drop(columns=['index']).sort_values('Insurance Count', ascending=False)

  # Map Data
  if state is None:
    dashboard['by_geo']=dict()
    by_geo = dashboard['by_geo']
    by_geo['name']='Insurance By State'

    params_states=dict()
    params_states['year']=year
    sql_states = f'''SELECT S1.geo_id AS 'State Code', g.name AS 'State', S1.counts AS 'Insurance Count', S1.amount AS 'Insurance Amount' FROM (SELECT geo_id, sum(i.count) as counts, sum(i.amount) as amount FROM insurance_agg i WHERE year=:year'''
    if quarter is not None:
      sql_states += ' AND quarter=:quarter'
      params_states['quarter']=quarter[0]

    sql_states += ''' GROUP BY geo_id)S1 JOIN geo g ON (S1.geo_id=g.id) WHERE geo_type='STA' '''

    stmt = sqlalchemy.text(sql_states,)
    df_states=pd.read_sql_query(stmt, conn, params=params_states)
    df_states['State']=df_states['State'].apply(convert_to_geojson_state)

    by_geo['charts']=list()
    by_geo['charts'].append({'title':'Insurance By State','data':df_states})
  return dashboard

def fetch_user_dash(conn, year,quarter, state=None):
  dashboard=dict()
  dashboard['object']='User'
  dashboard['title']='User Dashboard'
  dashboard['description']='Details of User and Devices for the selected timeline'

  # Process User and Device Totals
  dashboard['grand_total']=dict()
  grand=dashboard['grand_total']
  grand['name']='Registered Users'
  grand['charts']=list()

  dashboard['det_total']=dict()
  det=dashboard['det_total']
  det['name']='Registered Devices'
  det['charts']=list()

  # SQL Procesing
  sql= f'''SELECT u.reg_users as 'Users', u.app_opens as 'Application Opens' FROM user_agg u WHERE year=:year'''
  sql_device= f'''SELECT d.brand as 'Device Brand', sum(d.count) as 'Device Count', sum(d.percentage) as 'Device Share (%)' FROM device_agg d WHERE year=:year'''

  # Set the WHERE clause based on the fiters passed to this function
  params=dict()
  params['year']=year

  if quarter is not None:
    sql += ' AND quarter=:quarter'
    params['quarter']=quarter[0]

  sql += ' AND geo_id=:geo_id'
  if state is not None:
    params['geo_id']=state
  else:
    params['geo_id']=0

  sql_device += ' GROUP BY brand'

  stmt = sqlalchemy.text(sql,)
  df=pd.read_sql_query(stmt, conn,params=params)

  # Process the Totals
  usr_total=dict()
  usr_total['title']='Users'
  usr_total['value']=0

  app_total=dict()
  app_total['title']='Application Opens'
  app_total['value']=0

  for idx, row in df.iterrows():
    usr_total['value'] += row['Users']
    app_total['value'] += row['Application Opens']

  grand['charts'].append(usr_total)
  grand['charts'].append(app_total)

  # Process device totals
  stmt = sqlalchemy.text(sql_device,)
  df_device=pd.read_sql_query(stmt, conn,params=params)

  for idx, row in df_device.iterrows():
    dev_ct=dict()
    dev_ct['title']=row['Device Brand']
    dev_ct['value']=row['Device Count']
    dev_ct['metric']='Device Count'
    det['charts'].append(dev_ct)

    dev_pct=dict()
    dev_pct['title']=row['Device Brand']
    dev_pct['value']=row['Device Share (%)']
    dev_pct['metric']='Device Share (%)'
    det['charts'].append(dev_pct)

  # Execute top SQL
  if quarter is not None:
    dashboard['top']=dict()
    sql_top=f'''SELECT g.name as 'Geo', g.geo_type as 'Geo Type', u.reg_users as 'Users' FROM user_top u JOIN geo g ON (u.geo_id=g.id) WHERE year=:year AND quarter=:quarter AND top_in=:top_in'''

    params_top=dict()
    params_top['year']=year
    params_top['quarter']=quarter[0]

    if state is not None:
      sql_top += ' AND geo_id IN (SELECT id from geo WHERE parent_id=:parent_id)'
      params_top['top_in']='STA'
      params_top['parent_id']=state
    else:
      params_top['top_in']='CON'

    stmt = sqlalchemy.text(sql_top,)
    df_top=pd.read_sql_query(stmt, conn,params=params_top)

    # Populate 'top'
    top=dashboard['top']
    top['name']= 'Users By Top Geo'
    top['charts']=list()

    #if state is None:
    top['charts'].append(dict())
    top['charts'][0]['title']='Top States'
    top['charts'][0]['data']=df_top[df_top['Geo Type']=='STA'].reset_index().drop(columns=['index']).sort_values('Users', ascending=False)

    top['charts'].append(dict())
    top['charts'][1]['title']='Top Districts'
    top['charts'][1]['data']=df_top[df_top['Geo Type']=='DIS'].reset_index().drop(columns=['index']).sort_values('Users', ascending=False)

    top['charts'].append(dict())
    top['charts'][2]['title']='Top Pincodes'
    top['charts'][2]['data']=df_top[df_top['Geo Type']=='PIN'].reset_index().drop(columns=['index']).sort_values('Users', ascending=False)

  # Map Data
  if state is None:
    dashboard['by_geo']=dict()
    by_geo = dashboard['by_geo']
    by_geo['name']='User Info By State'

    params_states=dict()
    params_states['year']=year
    sql_states = f'''SELECT SQ.geo_id AS 'State Code', g.name AS 'State', SQ.users AS 'Users', SQ.apps AS 'App Opens'  FROM (SELECT geo_id, sum(u.reg_users) as users, sum(u.app_opens) as apps FROM user_agg u WHERE year=:year'''
    if quarter is not None:
      sql_states += ' AND quarter=:quarter'
      params_states['quarter']=quarter[0]

    sql_states += ''' GROUP BY geo_id)SQ JOIN geo g ON (SQ.geo_id=g.id) WHERE geo_type='STA' '''

    stmt = sqlalchemy.text(sql_states,)
    df_states=pd.read_sql_query(stmt, conn,params=params_states)
    df_states['State']=df_states['State'].apply(convert_to_geojson_state)

    by_geo['charts']=list()
    by_geo['charts'].append({'title':'User Info By State','data':df_states})

  return dashboard


def fetch_dash(conn,object,year,quarter=None,state=None):
  if object == 'Transaction':
    return fetch_txn_dash(conn, year,quarter, state)
  elif object=='Insurance':
    return fetch_ins_dash(conn, year, quarter,state)
  else:
    return fetch_user_dash(conn, year, quarter,state)


In [ ]:

fetch_user_dash(conn,2022,quarter=(1,'Q3'))

NameError: name 'conn' is not defined

# **GitHub Data Loader**

In [ ]:
'''
Load ALL data from JSON files in GDrive
'''
datadir='/content/drive/MyDrive/Guvi/Assignments/Phonepe/pulse/pulse/data'
print(f'''Loading data from {datadir}......20mins expected hold time''')
data=load_data(datadir)

# **GEO INDEXER**

In [ ]:
'''
Save Geo data to CSV files (States, Dis and Pin)
'''
geo = index_geo_info(datadir)

states = pd.DataFrame.from_dict(geo['states'],orient='index').reset_index().rename(columns={'index':'name', 0:'parent'})
districts = pd.DataFrame.from_dict(geo['districts'],orient='index').reset_index().rename(columns={'index':'name', 0:'parent'})
pincodes = pd.DataFrame.from_dict(geo['pincodes'],orient='index').reset_index().rename(columns={'index':'name', 0:'parent'})
states.name = states.name.astype(str)
districts.name = districts.name.astype(str)
pincodes.name = pincodes.name.astype(str)

states.to_csv('states.csv',index=False, sep=',', encoding='utf-8')
districts.to_csv('districts.csv',index=False, sep=',', encoding='utf-8')
pincodes.to_csv('pincodes.csv',index=False, sep=',', encoding='utf-8')

# **CSV Writers**

In [ ]:
'''
Write loaded Data to CSV files
'''

import os
import json
import pandas as pd
datadir='/content/drive/MyDrive/Guvi/Assignments/Phonepe/pulse/pulse/data'
#datadir='/content/drive/MyDrive/Learn/guvi/labs/Assignments/PhonePePulse/phonepe/pulse/data'


#hover_txns = pd.DataFrame(data['transaction']['map'])
#hover_users = pd.DataFrame(data['user']['map'])
#hover_ins = pd.DataFrame(data['insurance']['map'])

#print(f'''hover_txns - {hover_txns.shape}, hover_users - {hover_users.shape}, hover_ins - {hover_ins.shape}''')
#print(f'''{hover_txns.head().to_markdown()}''')
#print(f'''{hover_users.head().to_markdown()}''')
#print(f'''{hover_ins.head().to_markdown()}''')
# Write the dataframes into csv files
#hover_txns.to_csv('hover_txns.csv',index=False)
#hover_users.to_csv('hover_users.csv',index=False)
#hover_ins.to_csv('hover_ins.csv',index=False)


agg_txns = pd.DataFrame(data['transaction']['aggregated'])
agg_users = pd.DataFrame(data['user']['aggregated'])
agg_ins = pd.DataFrame(data['insurance']['aggregated'])
top_txns = pd.DataFrame(data['transaction']['top'])
top_users = pd.DataFrame(data['user']['top'])
top_ins = pd.DataFrame(data['insurance']['top'])
print(f'''agg_txns - {agg_txns.shape}, agg_users - {agg_users.shape}, agg_ins - {agg_ins.shape}''')
print(f'''top_txns - {top_txns.shape}, top_users - {top_users.shape}, top_ins - {top_ins.shape}''')

# Write the dataframes into csv files
agg_txns.to_csv('agg_txns.csv',sep=',', encoding='utf-8', index=False)
agg_users.to_csv('agg_users.csv',sep=',', encoding='utf-8', index=False)
agg_ins.to_csv('agg_ins.csv',sep=',', encoding='utf-8', index=False)
top_txns.to_csv('top_txns.csv',sep=',', encoding='utf-8',index=False)
top_users.to_csv('top_users.csv',sep=',', encoding='utf-8',index=False)
top_ins.to_csv('top_ins.csv',sep=',', encoding='utf-8',index=False)

agg_txns - (4619, 9), agg_users - (7215, 10), agg_ins - (590, 9)
top_txns - (17074, 9), top_users - (17075, 7), top_ins - (480, 9)


# **ENVIRONMENT CELLS**

In [ ]:
'''
  Configure Google Cloud Project Context
'''
project_id="<your GCP project id>"
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
'''
  Aunthenticate to Google Colab
'''
from google.colab import auth
auth.authenticate_user()


In [ ]:
!pip install cloud-sql-python-connector #package to connect SQL-Python in Cloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.33.0 which is incompatible.


In [ ]:
!pip install pymysql #Python pkg that creates API access to SQL

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.2 MB/s eta 0:00:00


# **DB CONNECTION CELL**

In [ ]:
'''
  Connect to Cloud SQL DB to populate data
'''

from pp_dbconnect import PPDbConnector

conname='<your db connection string>'
user= '<your user login>'
password='<your password>'
db='phonepe-bn'
pp_connector = PPDbConnector(conname,user,password,db)
conn = pp_connector.connect()


Successfully connected to 'phonepe-bn' database!! 


# **Test Dashboards Cell**

In [ ]:
fetch_user_dash(conn,2018,quarter=(1,'Q1'))

{'object': 'User',
 'title': 'User Dashboard',
 'description': 'Details of User and Devices for the selected timeline',
 'grand_total': {'name': 'Registered Users',
  'charts': [{'title': 'Users', 'value': 46877867},
   {'title': 'Application Opens', 'value': 0}]},
 'det_total': {'name': 'Registered Devices',
  'charts': [{'title': 'Apple', 'value': 20334366.0, 'metric': 'Device Count'},
   {'title': 'Apple', 'value': 0.0, 'metric': 'Device Share (%)'},
   {'title': 'Huawei', 'value': 9544952.0, 'metric': 'Device Count'},
   {'title': 'Huawei', 'value': 0.0, 'metric': 'Device Share (%)'},
   {'title': 'Lenovo', 'value': 13785649.0, 'metric': 'Device Count'},
   {'title': 'Lenovo', 'value': 0.0, 'metric': 'Device Share (%)'},
   {'title': 'Motorola', 'value': 18622946.0, 'metric': 'Device Count'},
   {'title': 'Motorola', 'value': 0.0, 'metric': 'Device Share (%)'},
   {'title': 'OnePlus', 'value': 15834429.0, 'metric': 'Device Count'},
   {'title': 'OnePlus', 'value': 0.0, 'metric': 'D

In [ ]:
'''
Save Geo Data into DB
'''
# Connect to Cloud SQL DB to populate data
import traceback
from pp_dbconnect import PPDbConnector

conname='<your db connection string>'
user= '<your sql login>'
password='<your sql password>'
db='phonepe-bn'
pp_connector = PPDbConnector(conname,user,password,db)
conn = pp_connector.connect()
import pandas as pd
states = pd.read_csv('states.csv')
districts = pd.read_csv('districts.csv')
pincodes = pd.read_csv('pincodes.csv')

print(f'''states - {states.shape}, districts - {districts.shape}, pincodes - {pincodes.shape}''')

try:
  save_geo(states,districts,pincodes,conn)
  print('Saved Geo information in DB successfully !!')
except:
  print('''Failed to save 'geo' information - Ensure 'geo' table is empty before running this !!''')
  print(traceback.print_exc())

Successfully connected to 'phonepe-bn' database!! 
states - (36, 2), districts - (417, 2), pincodes - (1115, 2)
Saved Geo information in DB successfully !!


In [ ]:
'''
Populate Transaction Data into DB
'''
import pandas as pd
import traceback
agg_txns=pd.read_csv('agg_txns.csv')
top_txns=pd.read_csv('top_txns.csv')

print(f'''agg_txns - {agg_txns.shape}, top_txns - {top_txns.shape}''')

try:
  save_txns(agg_txns,top_txns,conn)
  print('Saved Transaction information in DB successfully !!')
except:
  print('''Failed to save 'transaction' information - Ensure 'transaction' table is empty before running this !!''')
  print(traceback.print_exc())

agg_txns - (4619, 9), top_txns - (17074, 9)
geo - (1569, 4)
geo_type = PIN, geo_parent = ladakh
geo_type = PIN, geo_parent = ladakh
agg_txns(db): (4619, 8), top_txns(db):(17074, 8)
Saved Transaction information in DB successfully !!


In [ ]:
'''
Populate Insurance Data into DB
'''
import pandas as pd
import traceback
agg_ins=pd.read_csv('agg_ins.csv')
top_ins=pd.read_csv('top_ins.csv')

print(f'''agg_ins - {agg_ins.shape}, top_ins - {top_ins.shape}''')

try:
  save_ins(agg_ins,top_ins,conn)
  print('Saved Insurance information in DB successfully !!')
except:
  print('''Failed to save 'insurance' information - Ensure 'insurance' table is empty before running this !!''')
  print(traceback.print_exc())

agg_ins - (590, 9), top_ins - (480, 9)
geo - (1569, 4)
agg_ins(db): (590, 8), top_ins(db):(480, 8)
Saved Insurance information in DB successfully !!


In [ ]:
'''
Populate USERS Data into DB
'''
import pandas as pd
import traceback
agg_users=pd.read_csv('agg_users.csv')
top_users=pd.read_csv('top_users.csv')

print(f'''agg_users - {agg_users.shape}, top_users - {top_users.shape}''')

try:
  save_users(agg_users,top_users,conn)
  print('Saved USERS information in DB successfully !!')
except:
  print('''Failed to save 'users' information - Ensure 'users' tables are empty before running this !!''')
  print(traceback.print_exc())

agg_users - (7215, 10), top_users - (17075, 7)
geo - (1569, 4)
agg_users(db): (7215, 9), top_users(db):(17075, 6)


<ipython-input-81-ff65c745f81a>:458: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  u_agg.drop_duplicates(inplace=True,ignore_index=True)


user_agg(db): (925, 7), device_agg(db): (7215, 7), top_users(db):(17075, 7)
Saved USERS information in DB successfully !!


In [ ]:
top_txns[(top_txns.year==2018) & (top_txns.quarter==1) & (top_txns.geo_id==102)].reset_index()

,index,year,quarter,top_in,stat_type,count,amount,geo_id
0,3353,2018,1,STA,TOTAL,158509,1.698650e+08,102
